In [1]:
#import libraries

import scipy.sparse as sps
import numpy as np
import os
import pandas as pd 

URM_all= sps.load_npz("./Processed Matrices/URM_simple_coo_2020.npz").tocsr()
ICM_all= sps.load_npz("./Processed Matrices/ICM_simple_coo_2020.npz").tocsr()

In [2]:
%load_ext Cython

In [3]:
from Notebooks_utils.evaluation_function import evaluate_algorithm
from Base.Similarity.Compute_Similarity_Python import Compute_Similarity_Python
from Base.Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

np.random.seed(41148)
URM_train, URM_valid = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)

In [4]:
from GraphBased.RP3betaRecommender import RP3betaRecommender
RP3brecommender= RP3betaRecommender(URM_train)
RP3brecommender.fit(topK=770, alpha=0.414615342821205, beta=0.17823717015919388, normalize_similarity=False)

RP3betaRecommender: URM Detected 82 (1.03 %) cold users.
RP3betaRecommender: URM Detected 2372 (9.13 %) cold items.


In [5]:
from GraphBased.P3alphaRecommender import P3alphaRecommender
P3arecommender= P3alphaRecommender(URM_train)
P3arecommender.fit(topK=267, alpha=0.4079116116383131, normalize_similarity=False)

P3alphaRecommender: URM Detected 82 (1.03 %) cold users.
P3alphaRecommender: URM Detected 2372 (9.13 %) cold items.


In [7]:
from KNN.ItemKNNSimilarityHybridRecommender import ItemKNNSimilarityHybridRecommender

recommender=ItemKNNSimilarityHybridRecommender(URM_train, RP3brecommender.W_sparse, P3arecommender.W_sparse)
evaluator_validation = EvaluatorHoldout(URM_valid, cutoff_list=[10])
resultDF= pd.DataFrame({"topK":[],"alpha":[],"MAP":[]})
for alpha in range(0,101,5):
    for topK in range(50,1201,50):
        recommender.fit(topK,alpha/100)
        MAP,_=evaluator_validation.evaluateRecommender(recommender)
        MAP=MAP[10]["MAP"]
        resultDF=resultDF.append({"topK":[topK],"alpha":[alpha/100],"MAP":[MAP]},ignore_index=True)
        print(topK,alpha/100, MAP)

ItemKNNSimilarityHybridRecommender: URM Detected 82 (1.03 %) cold users.
ItemKNNSimilarityHybridRecommender: URM Detected 2372 (9.13 %) cold items.
EvaluatorHoldout: Processed 5657 ( 100.00% ) in 3.24 sec. Users per second: 1744
50 0.0 0.058533824128778035
EvaluatorHoldout: Processed 5657 ( 100.00% ) in 3.27 sec. Users per second: 1729
100 0.0 0.06079158074133204
EvaluatorHoldout: Processed 5657 ( 100.00% ) in 3.31 sec. Users per second: 1709
150 0.0 0.06064857373854543
EvaluatorHoldout: Processed 5657 ( 100.00% ) in 3.32 sec. Users per second: 1704
200 0.0 0.06007728094183866
EvaluatorHoldout: Processed 5657 ( 100.00% ) in 3.46 sec. Users per second: 1637
250 0.0 0.06026125830654245
EvaluatorHoldout: Processed 5657 ( 100.00% ) in 3.42 sec. Users per second: 1656
300 0.0 0.06069687873578126
EvaluatorHoldout: Processed 5657 ( 100.00% ) in 3.38 sec. Users per second: 1675
350 0.0 0.06069687873578126
EvaluatorHoldout: Processed 5657 ( 100.00% ) in 3.35 sec. Users per second: 1687
400 0.0 

In [8]:
resultDF_sorted=resultDF.sort_values(by=['MAP'],ascending=False)
resultDF_sorted.to_csv("Hybrid_P3a_RP3b_results_sorted.csv")

In [9]:
RP3brecommender_full= RP3betaRecommender(URM_all)
RP3brecommender_full.fit(topK=770, alpha=0.414615342821205, beta=0.17823717015919388, normalize_similarity=False)

P3arecommender_full= P3alphaRecommender(URM_all)
P3arecommender_full.fit(topK=267, alpha=0.4079116116383131, normalize_similarity=False)

recommender_full= ItemKNNSimilarityHybridRecommender(URM_all, RP3brecommender_full.W_sparse, P3arecommender_full.W_sparse)
recommender_full.fit(topK=resultDF_sorted.iloc[0]['topK'][0], alpha=resultDF_sorted.iloc[0]['alpha'][0])

RP3betaRecommender: URM Detected 1079 (4.15 %) cold items.
P3alphaRecommender: URM Detected 1079 (4.15 %) cold items.
ItemKNNSimilarityHybridRecommender: URM Detected 1079 (4.15 %) cold items.


In [10]:
rec_df=pd.DataFrame({"user_id":[],"item_list":[]})
target_user_list=list(pd.read_csv("./Data/2020/data_target_users_test.csv").user_id)

for uid in target_user_list:
    rec_list=map(str,recommender_full.recommend(uid)[:10])
    rec_df=rec_df.append({"user_id":uid,"item_list":' '.join(rec_list)},ignore_index=True)

    
rec_df=rec_df.astype({"user_id":'int32',"item_list":'str'})
rec_df=rec_df.set_index('user_id')
rec_df.to_csv("Hybrid_P3a_RP3b_recommendations.csv")